<a href="https://colab.research.google.com/github/ykitaguchi77/Kaggle/blob/main/H-M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Open transaction file**

In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Deep_learning/Kaggle_H&M/transactions_train.csv')

In [4]:
#Pickle形式で一旦保存（Loadが早くなる）
df.to_pickle('/content/drive/MyDrive/Deep_learning/Kaggle_H&M/transactions_train.pickle') # pickle ファイルに変換

In [ ]:
train = pd.read_pickle('/content/drive/MyDrive/Deep_learning/Kaggle_H&M/transactions_train.pickle') # pickleファイルの読み込み
train

#**重複を削除したカスタマーリストを作成**

In [19]:
customer_id_list = train.iloc[:, 1]
customer_id_list = customer_id_list[~customer_id_list.duplicated()].reset_index(drop=True)
customer_id_list

0          000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
1          00007d2de826758b65a93dd24ce629ed66842531df6699...
2          00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...
3          0008968c0d451dbc5a9968da03196fe20051965edde741...
4          000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...
                                 ...                        
1362276    fe99a0069d6b3c64c2707d0ce53b9311540917471d82df...
1362277    fecc5f77b5f7ee4570efde9ab05ec94d0de2bf80efb4f6...
1362278    fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...
1362279    fee56cc5315dafb35a4490ccc6f711092cae913550c832...
1362280    ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...
Name: customer_id, Length: 1362281, dtype: object

In [16]:
print(len(customer_id_list)) #1362281人のデータ

1362281


In [29]:
a = train[train.iloc[:,1] == customer_id_list[3]]　 #customerごとの購入履歴
len(a)

61

#**重複を削除した商品リストを作成**

In [34]:
article_list = train.iloc[:, 2]
article_list = article_list[~article_list.duplicated()].reset_index(drop=True)
article_list #商品が104546個もある！！

0         663713001
1         541518023
2         505221004
3         685687003
4         685687004
            ...    
104542    913290001
104543    860135008
104544    539060028
104545    810737015
104546    533261032
Name: article_id, Length: 104547, dtype: int64

In [35]:
len(article_list)

104547

#**類似商品のクラスタリング**
https://qiita.com/kotai2003/items/55c4f525ff216ca16800

In [ ]:
!pip install pycaret
import pycaret
from pycaret.datasets import get_data


In [ ]:
import pandas as pd
import pandas_profiling
article = pd.read_csv("/content/drive/MyDrive/Deep_learning/Kaggle_H&M/articles.csv")
article = article[["product_type_name","product_group_name","graphical_appearance_name", "colour_group_name","perceived_colour_value_name", "perceived_colour_master_name", "department_name", "index_name", "garment_group_name"]]
article.profile_report()

In [3]:
data = article.sample(frac=0.95, random_state=786).reset_index(drop=True)
data_unseen = article.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (100265, 9)
Unseen Data For Predictions: (5277, 9)


In [5]:
from pycaret.clustering import *
data_clust = setup(data, normalize = True, session_id = 123)

,Description,Value
0,session_id,123
1,Original Data,"(100265, 9)"
2,Missing Values,False
3,Numeric Features,0
4,Categorical Features,9
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(100265, 539)"
9,CPU Jobs,-1


In [ ]:
kmeans = create_model('kmeans',num_clusters = 10 )
print(kmeans)

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:16:26
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Clusters
Estimator,. . . . . . . . . . . . . . . . . .,K-Means Clustering


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness


In [ ]:
kmean_results = assign_model(kmeans)
kmean_results.head()

In [ ]:
#PCA plot
plot_model(kmeans)

In [ ]:
#Elbow plot (suggested cluster number)
plot_model(kmeans, plot = 'elbow')

In [ ]:
#Distribution plot
plot_model(kmeans, plot = 'distribution', feature = 'class')

In [ ]:
#Predictions
unseen_predictions = predict_model(kmeans, data=data_unseen)
unseen_predictions.head()